In [42]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from datetime import datetime,timedelta
import yfinance as yf 
from nselib import capital_market 
import os 
from lightweight_charts import Chart
import vectorbt as vbt 
from tabulate import tabulate 
from tqdm import tqdm
import talib as ta 
import time 
sns.set_style('darkgrid')
plt.style.use('ggplot')

In [43]:
os.getcwd()
if not os.path.exists("data"):
    os.mkdir('data')
else:
    print("Folder present")

Folder present


In [44]:
#getting All symbols
nifty_50=capital_market.nifty50_equity_list()['Symbol']
a=[[i] for i in nifty_50]
head=['index','symbols']
nf=[['ayush'],['motusu'],['harshit'],['nani'],['mami']]
print(tabulate(a, headers=head, tablefmt="fancy_grid",showindex=True))


╒═════════╤════════════╕
│   index │ symbols    │
╞═════════╪════════════╡
│       0 │ ADANIENT   │
├─────────┼────────────┤
│       1 │ ADANIPORTS │
├─────────┼────────────┤
│       2 │ APOLLOHOSP │
├─────────┼────────────┤
│       3 │ ASIANPAINT │
├─────────┼────────────┤
│       4 │ AXISBANK   │
├─────────┼────────────┤
│       5 │ BAJAJ-AUTO │
├─────────┼────────────┤
│       6 │ BAJFINANCE │
├─────────┼────────────┤
│       7 │ BAJAJFINSV │
├─────────┼────────────┤
│       8 │ BPCL       │
├─────────┼────────────┤
│       9 │ BHARTIARTL │
├─────────┼────────────┤
│      10 │ BRITANNIA  │
├─────────┼────────────┤
│      11 │ CIPLA      │
├─────────┼────────────┤
│      12 │ COALINDIA  │
├─────────┼────────────┤
│      13 │ DIVISLAB   │
├─────────┼────────────┤
│      14 │ DRREDDY    │
├─────────┼────────────┤
│      15 │ EICHERMOT  │
├─────────┼────────────┤
│      16 │ GRASIM     │
├─────────┼────────────┤
│      17 │ HCLTECH    │
├─────────┼────────────┤
│      18 │ HDFCBANK   │


In [45]:
class download_symb:
    def __init__(self,symbol):
        self.symbol=symbol
    def symb(self):
        end_date=datetime.now().date()
        start_date=end_date-timedelta(days=7)
        try:
            #data_sym=yf.download(f'{self.symbol}.NS',start=start_date,end=end_date,interval="1m")
            #self.data=data_sym.to_parquet(f'data/{self.symbol}.parquet')
            self.data=pd.read_parquet(f'data/{self.symbol}.parquet')            
        except Exception as e:
            print("File not downloaded")
        return(self.data)
        

In [46]:
data=download_symb(nifty_50[0])
data.symb()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-02-22 09:15:00+05:30,3234.000000,3237.850098,3220.000000,3236.800049,3236.800049,0
2024-02-22 09:16:00+05:30,3236.949951,3239.399902,3233.399902,3237.850098,3237.850098,11415
2024-02-22 09:17:00+05:30,3238.899902,3241.949951,3236.000000,3240.000000,3240.000000,13270
2024-02-22 09:18:00+05:30,3239.350098,3242.000000,3239.100098,3241.649902,3241.649902,9177
2024-02-22 09:19:00+05:30,3241.600098,3242.949951,3234.600098,3236.899902,3236.899902,11502
...,...,...,...,...,...,...
2024-02-28 15:25:00+05:30,3211.149902,3211.850098,3210.000000,3211.000000,3211.000000,2751
2024-02-28 15:26:00+05:30,3211.000000,3211.000000,3209.100098,3211.000000,3211.000000,4867
2024-02-28 15:27:00+05:30,3211.000000,3215.500000,3208.699951,3214.850098,3214.850098,3610


In [47]:
for i in tqdm(range(0,len(nifty_50))):
    try:
        data=download_symb(nifty_50[i])
        data.symb()
    except Exception as e:
        continue

100%|██████████| 50/50 [00:00<00:00, 140.59it/s]
